In [ ]:
import torch
import numpy as np
import scipy.integrate
from numpy import sin, cos
import matplotlib.pyplot as plt
import random
solver = scipy.integrate.solve_ivp

seed = 12
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.determinstic = True

In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
import pandas as pd

BATCH_size = 1000

dftarget = pd.read_csv("./data/target.csv", header=None, dtype=np.float32)
dfinput = pd.read_csv("./data/input.csv", header=None, dtype=np.float32)

X_train, X_test, Y_train, Y_test = train_test_split(dfinput.values, dftarget.values, test_size=0.2)

# train data
data_train = data_utils.TensorDataset(torch.tensor(X_train), torch.tensor(Y_train))
train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_size, shuffle=True)

# test data
data_test = data_utils.TensorDataset(torch.tensor(X_test), torch.tensor(Y_test))
test_loader = torch.utils.data.DataLoader(data_test, batch_size=BATCH_size, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#!nvidia-smi

In [ ]:
class MLP(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(MLP,self).__init__()
    self.l1 = nn.Linear(input_size,hidden_size)
    self.l2 = nn.Linear(hidden_size,hidden_size)
    self.l3 = nn.Linear(hidden_size,output_size)
  
  def forward(self,x):
    x = self.l1(x)
    x = torch.tanh(x)
    x = self.l2(x)
    x = torch.tanh(x)
    x = self.l3(x)
    return x

  def grad(self,x):
    x = x.requires_grad_(True)
    with torch.enable_grad():
      hamiltonian = self(x)
      gradient = torch.autograd.grad(hamiltonian.sum(),x,create_graph=True,retain_graph=True)
    return gradient[0]

  def fvec(self,t,x):
    return self.grad(x)

In [ ]:
class MLPS(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MLPS, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = self.l1(x)
    x = torch.tanh(x)
    x = self.l2(x)
    x = torch.tanh(x)
    x = self.l3(x)
    return x

In [ ]:
import time
input_size = 2
hidden_size = 200
output_size = 1

input_size_s = 2
hidden_size_s = 200
output_size_s = 4

num_trials = 10
stats = {'train_loss': [], 'eval_loss': [], 'computation_time': []}

for trial in range(num_trials):
  h_nn = MLP(input_size, hidden_size, output_size).to(device)
  s_nn = MLPS(input_size_s, hidden_size_s, output_size_s).to(device)
  num_epochs = 2000
  parmas = list(h_nn.parameters()) + list(s_nn.parameters())
  criterion = nn.MSELoss()
  optimizer = optim.Adam(params=parmas,lr=0.001)

  history_loss = []
  history_eval = []
  history_acc = []
  startt = time.time()
  for epoch in range(num_epochs):
    estartt = time.time()
    h_nn.train()
    s_nn.train()

    total_loss = 0.0
    eval_loss = 0.0
    cnt = 0
    for i,(data,target) in enumerate(train_loader):
      optimizer.zero_grad()
      x = data.to(device)
      output = h_nn.grad(x)
      skew_init = s_nn.forward(x)
      skew_n = torch.zeros(skew_init.shape[0],2,2).to(device)
      skew_n[:,0,1] = skew_init[:,0]
      skew_n[:,1,0] = -skew_init[:,0]

      
      output = output.unsqueeze(2)
      output = torch.matmul(skew_n, output)
      output = output.squeeze(2)

      target = target.to(device)

      loss = criterion(output,target)
      loss.backward()
      optimizer.step()
     

      total_loss = total_loss+loss.cpu().item()
      cnt = cnt + 1
    total_loss = total_loss/cnt
  
    num_correct = 0
    num_data = 0
    h_nn.eval()
    s_nn.eval()
    eval_loss = 0.0
    cnt = 0
    for i,(data,target) in enumerate(test_loader):
      optimizer.zero_grad()
      x = data.to(device)
      output = h_nn.grad(x)
      skew_init = s_nn.forward(x)
      skew_n = torch.zeros(skew_init.shape[0],2,2).to(device)
      skew_n[:,0,1] = skew_init[:,0]
      skew_n[:,1,0] = -skew_init[:,0]

      
      output = output.unsqueeze(2)
      output = torch.matmul(skew_n, output)
      output = output.squeeze(2)

      eval_loss = eval_loss+criterion(output,target.to(device)).cpu().item()
      cnt = cnt + 1
    eval_loss = eval_loss/cnt

    history_loss.append(total_loss)
    history_eval.append(eval_loss)
  print("{}/{} training loss:{},evaluation loss:{}".format(epoch+1,num_epochs,total_loss,eval_loss))
  path = "./model/skew_h_lv_v1{}".format(trial)
  torch.save(h_nn.state_dict(), path)
  path = "./model/skew_s_lv_v1{}".format(trial)
  torch.save(s_nn.state_dict(), path)
  stats['train_loss'].append(total_loss)
  stats['eval_loss'].append(eval_loss)
  stats['computation_time'].append(time.time() - startt)

print("train_loss:{}, std: {}".format(np.mean(stats['train_loss']), np.std(stats['train_loss'])))
print("test_loss:{}, std: {}".format(np.mean(stats['eval_loss']), np.std(stats['eval_loss'])))
print("computation_time:{}, std: {}".format(np.mean(stats['computation_time']), np.std(stats['computation_time'])))

In [ ]:
A = pd.read_csv("./data/A.csv", header=None, dtype=np.float32)
B = pd.read_csv("./data/B.csv", header=None, dtype=np.float32)
A = np.mat(A)
B = np.mat(B)

In [ ]:
# simulation
# parameters
a11 = 1.0
a12 = -1.0
a21 = -1.0
a22 = 1.0
eq_error = []
net_error = []
for trial in range(num_trials):
  s_nn.load_state_dict(torch.load("./model/skew_s_lv_v1{}".format(trial), map_location=device))
  s_nn.eval()
  h_nn.load_state_dict(torch.load("./model/skew_h_lv_v1{}".format(trial), map_location=device))
  h_nn.eval()
  teval = torch.linspace(0.0,30.0,100)
  x0 = np.random.uniform(low=0.0, high=1.0, size=2)
  #x0 = (0.41527075, 0.135658)
  print(x0)
  dt = teval[1]-teval[0]

  
  teval = teval.detach().cpu().numpy()
  A = torch.tensor(A, dtype=torch.float32)
  B = torch.tensor(B, dtype=torch.float32)
  h_nn.cpu()
  s_nn.cpu()
  def fvec_np(x, t):
      tx = torch.tensor(x, dtype=torch.float).unsqueeze(0)
      x = torch.matmul(tx, torch.inverse(B))
      output = h_nn.grad(x)
      skew_init = s_nn.forward(x)
      skew_n = torch.zeros(skew_init.shape[0],2,2)
      skew_n[:,0,1] = skew_init[:,0]
      skew_n[:,1,0] = -skew_init[:,0]


      output = output.unsqueeze(2)
      output = torch.matmul(skew_n, output)
      output = torch.matmul(A, output)
      output = output.squeeze(2)

      output = output.squeeze(0)
      return output.detach().cpu().numpy()
  res = scipy.integrate.odeint(fvec_np, x0, teval)

  ax=plt.subplot()
  ax.set_ylim([0,4])
  ax.plot(teval,res[:,0])
  ax.plot(teval,res[:,1])
  plt.show()
  def net_energy(x,t):
    tx = torch.tensor(x, dtype=torch.float)
    x = torch.matmul(tx, torch.inverse(B))
    ne = h_nn(x).squeeze(0)
    return ne.detach().cpu().numpy()

  net_energy = net_energy(res, teval).reshape((100))
  plt.plot(teval,net_energy)
  plt.show()
  net_energy_error=net_energy[99]-net_energy[0]
  print("energy_error{}:{}".format(trial,net_energy_error))
    
  energy = -a21*np.log(res[:,0])-a22*res[:,0]+a11*np.log(res[:,1])+a12*res[:,1]
  energy_error=energy[99]-energy[0]
  print("energy_error{}:{}".format(trial,energy_error))
  plt.plot(energy)
  plt.show()

  eq_error.append(energy_error)
  net_error.append(net_energy_error)
print("eq_error:{}".format(eq_error),"net_error:{}".format(net_error))
print("eq_error:{}, std: {}".format(np.mean(np.abs(eq_error)), np.std(eq_error)))
print("net_error:{}, std: {}".format(np.mean(np.abs(net_error)), np.std(net_error)))


In [ ]:
# Ground truth
# parameters
a11 = 1.0
a12 = -1.0
a21 = -1.0
a22 = 1.0

def func(t, state):
  dvdt = np.zeros_like(state)
  dvdt[0] = a11*state[0] + a12*state[0]*state[1]
  dvdt[1] = a21*state[1] + a22*state[0]*state[1]
  return dvdt

#　the time grid
M = 100
tend = 30.0
t_eval = np.linspace(0,tend,M)
dt = t_eval[1]-t_eval[0]
num = 1000
state = []
for i in range(1):
  s = (0.41527075, 0.1356582)
  state.append(s)

q1 = []
q2 = []

flag = False
for i in range(1):
  sol = solver(func, [0, tend], state[i], t_eval=t_eval)
  tval = sol['t']
  q1 = sol['y'][0]
  q2 = sol['y'][1]
  plt.plot(tval, q1, 'steelblue')
  plt.plot(tval, q2, 'orange')
  plt.show()